In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load Data

In [3]:
train=pd.read_csv('/content/drive/MyDrive/kag fellow/train.csv')
test=pd.read_csv('/content/drive/MyDrive/kag fellow/test.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth',None)

### Extracting Info From Engine,Transmission and reconstruct fuel_types

In [8]:
def extracting_info_from_df(df):

    # Extracting info from engine column

    # Extracting HP
    df['power']=df['engine'].apply( lambda x:  re.findall(r"(\d+).+HP" ,x)).explode().astype('float')

    # Extracting engine size
    df['engine_size']=df.engine.apply(lambda x: re.findall(r"(\d+.\d+)[L]",x)).explode().astype('float')
    df.loc[ df.engine.str.contains('liter',case=False),'engine_size']=df.engine.apply(lambda x: re.findall(r"(\d+.\d+).+Liter",x)).explode().astype('float')

    # Extracting engine cyliders
    df['engine_cylinders']=df.engine.apply(lambda x: re.findall(r"(\d+).[Cylinder]",x )).explode().astype('float')
    df.loc[df.engine_cylinders.isnull(),'engine_cylinders']=df.engine.apply(lambda x: re.findall(r"[V|I|H](\d)",x)).explode().astype('float')

    # Extracting turbo engines
    df['turbo']=df.engine.apply(lambda x: 2 if 'twin' in x.lower() else (1 if 'turbo' in x.lower() else 0))
    df['Supercharged']=df.engine.apply(lambda x: 1 if 'Supercharged' in x.lower() else 0)

    # Extracting additional info from engine
    df['GDI']=df.engine.apply(lambda x: 1 if 'gdi' in x.lower() else 0)
    df['PDI']=df.engine.apply(lambda x: 1 if 'pdi' in x.lower() else 0)
    df['DOHC']=df.engine.apply(lambda x: 1 if 'dohc' in x.lower().split() else 0)
    df['SOHC']=df.engine.apply(lambda x: 1 if 'sohc'  in x.lower()  else 0)
    df['OHV']=df.engine.apply(lambda x: 1 if 'ohv'  in x.lower()  else 0)
    df['MPFI']=df.engine.apply(lambda x: 1 if 'mpfi'  in x.lower()  else 0)
    df['DDI']=df.engine.apply(lambda x: 1 if 'ddi'  in x.lower()  else 0)
    df['TFSI']=df.engine.apply(lambda x: 1 if 'tfsi'  in x.lower()  else 0)

    # Extracting engine gas engine cylinder mountain
    df['W_engine']=df.engine.apply(lambda x: 1 if 'W' in x else 0)
    df['Rotary_engine']=df.engine.apply(lambda x: 1 if 'rotary' in x.lower() else 0)
    df['Flat_engine']=df.engine.apply(lambda x: 1 if 'flat' in x.lower() else 0)
    df['H_engine']=df.engine.str.contains(r'\bh\d\b',case=False).astype('int')
    df['V_engine']=df.engine.str.contains(r'\bv\d\b',case=False).astype('int')
    df['I_engine']=df.engine.str.extract(r'\b(I|straight)-?\d?\b',flags=re.IGNORECASE,expand=False).notna().astype('int')
    df['E_engine']=df.engine.apply(lambda x: 1 if 'electric' in x.lower() else 0)



    # Extracting info From Transmission column

    # Extracting number of speeds
    df['number_of_speeds']=df['transmission'].apply(lambda x : re.findall(r"\d+",x)).explode().astype('float')

    # Extracting all other info from transmission
    df['A/T']=df.transmission.str.contains('A/T|automatic',case=False).astype('int')
    df['M/T']=df.transmission.str.contains('M/T|manual|mt',case=False).astype('int')
    df['CVT']=df.transmission.apply(lambda x: 1 if 'cvt' in x else 0)
    df['Dual_Shift']=df.transmission.apply(lambda x: 1 if 'w/Dual' in x else 0)
    df['Auto-Shift']=df.transmission.apply(lambda x : 1 if 'Auto-Shift' in x else 0)
    df['Overdrive']=df.transmission.apply(lambda x : 1 if 'Auto-Overdrive' in x else 0)
    df['DCT']=df.transmission.apply(lambda x : 1 if 'DCT' in x else 0)
    df['Automatic with O']=df.transmission.apply(lambda x : 1 if 'Controlled Automatic with O' in x else 0)
    df['SCHEDULED FOR OR IN PRODUCTION']=df.transmission.apply(lambda x : 1 if 'SCHEDULED FOR OR IN PRODUCTION' in x else 0)
    df['Variable']=df.transmission.apply(lambda x : 1 if 'Variable' in x else 0)
    df['At/Mt']=df.transmission.str.contains('At/Mt',case=False).astype('int')
    df['F']=0
    df.loc[df.transmission=='F','F']=1

    # Correct fuel_types
    df.loc[(df.engine.str.contains('diesel',case=False))&(df.fuel_type!='Diesel'),'fuel_type']='Diesel'
    df.loc[(df.engine.str.contains('Plug-In',case=False))&(df.fuel_type!='Plug-In Hybrid'),'fuel_type']='Plug-In Hybrid'
    df.loc[df.engine.str.contains('hybrid',case=False)&(~df.engine.str.contains('Plug-In',case=False))&((df.fuel_type!='Hybrid')),'fuel_type']='Hybrid'
    df.loc[df.engine.str.contains('electric',case=False)&(df.fuel_type=="Gasoline"),'fuel_type']='Electric'
    df.loc[df.brand=='Tesla','fuel_type']='Electric'
    df.loc[(df.engine.str.contains('flexible|flex',case=False))&(df.fuel_type!='E85 Flex Fuel'),'fuel_type']='E85 Flex Fuel'
    df.loc[(df.engine.str.contains('hydrogen',case=False)),'fuel_type']='Hydrogen'
    df.loc[(df.engine.str.contains('electric',case=False))&(~df.engine.str.contains('gas',case=False))&(~df.engine.str.contains('hydro',case=False))&(df.fuel_type!='Electric'),'fuel_type']='Electric'

    return df


In [12]:
# Train Extracting Features
train= extracting_info_from_df(train)
train.loc[(train.I_engine==1)&(train.V_engine==1),'I_engine']=0

In [13]:
# Test Extracting Features
test=extracting_info_from_df(test)
test.loc[(test.I_engine==1)&(test.V_engine==1),'I_engine']=0

In [ ]:
# df=df[df.model.isin(test.model)]

### Creating new features

In [14]:
train['mile_per_year']=train['milage']/(2024-train['model_year'])
test['mile_per_year']=test['milage']/(2024-test['model_year'])
train['bb']=[1 if train.ext_col[i]==train.int_col[i] else 0 for i in range(len(train))]
test['bb']=[1 if test.ext_col[i]==test.int_col[i] else 0 for i in range(len(test))]
train['cat_mil']=train['milage'].apply(lambda x: 5 if x<=5000 else 4 if x<=20000 else 3 if x<=50000 else 2 if x<=100000 else 1)
test['cat_mil']=test['milage'].apply(lambda x: 5 if x<=5000 else 4 if x<=20000 else 3 if x<=50000 else 2 if x<=100000 else 1)
train['cn']=train.groupby(['brand','model','model_year'])['price'].transform('count')
col1=train.pop('cn')
train.insert(13, 'cn', col1)
test['cn']=test.groupby(['brand','model','model_year'])['milage'].transform('count')
col2=test.pop('cn')
test.insert(13, 'cn', col2)

In [ ]:
train.to_csv('/content/drive/MyDrive/kag fellow/train_pre.csv')
test.to_csv('/content/drive/MyDrive/kag fellow/train_pre.csv')